# Addition (Sum) Prediction Problem

In [40]:
from random import randint
from numpy import array
from math import ceil
from math import log10
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

### 1 Generate Sum Pairs
The first step is to generate sequences of random integers and their sum. We can put this in a function named random sum pairs(), as follows.

1. n terms: The number of terms in the equation, (e.g. 2 for 10+10).
2. largest: The largest numerical value for each term (e.g. 10 for values between 1-10).
3. alphabet: The symbols used to encode the input and output sequences (e.g. 0-9, + and ‘ ’)

In [41]:
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
  X, y = list(), list()
  for _ in range(n_examples):
    in_pattern = [randint(1,largest) for _ in range(n_numbers)]
    out_pattern = sum(in_pattern)
    X.append(in_pattern)
    y.append(out_pattern)
    return X,y          

In [42]:
#------------For Seeing the output--------# 
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)        

[[3, 8]] [11]


### 2 Integers to Padded Strings

The next step is to convert the integers to strings. The input string will have the format ‘10+10’ and the output string will have the format ‘20’. 

Key to this function is the padding of numbers to ensure that each input and output sequence has the same number of characters. A padding character should be different from the data so the model can learn to ignore them. 

In this case, we use the space character for padding(‘ ’) and pad the string on the left, keeping the information on the far right.

In [43]:
# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = 5
    Xstr = list()
    for pattern in X:
        strp =  '+' .join([str(n) for n in pattern])
        strp =   ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    max_length = int(ceil(log10(n_numbers * (largest+1))))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp =   ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr


In [44]:
#Example of output of converting a sequence pair to padded characters.
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)

[[8, 9]] [17]
['  8+9'] ['17']


### 3 Integer Encoded Sequences

Next, we need to encode each character in the string as an integer value. 

We have to work with numbers in neural networks after all, not characters. 

Integer encoding transforms the problem into a classification problem where the output sequence may be considered class outputs with 11 possible values each. 

To perform this encoding, we must define the full alphabet of symbols that may appear in the string encoding, as follows: alphabet=['0','1','2','3','4','5','6','7','8','9','+','']

<b>Integer encoding then becomes a simple process of building a lookup table of character-to- integer offset and converting each char of each string, one by one. </b>
The example below provides the <b>integer_encode()</b> function for integer encoding and demonstrates how to use it.

<img src='intenc.png'>

In [45]:
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [46]:
#Example of integer encoding padded sequences.
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode 
alphabet=['0','1','2','3','4','5','6','7','8','9','+',' '] 
X, y = integer_encode(X, y, alphabet)
print(X, y)


[[3, 5]] [8]
['  3+5'] [' 8']
[[11, 11, 3, 10, 5]] [[11, 8]]


### 4 One Hot Encoded Sequences

The next step is to binary encode the integer encoding sequences. This involves converting each integer to a binary vector with the same length as the alphabet and marking the specific integer with a 1. 

For example, a 0 integer represents the ‘0’ character and would be encoded as a binary vector with a 1 in the 0th position of an 11 element vector: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]. 

The example below defines the <b>one_hot_encode()</b> function for binary encoding and demonstrates how to use it.

In [47]:
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
    for index in seq:
        vector = [0 for _ in range(max_int)]
        vector[index] = 1
        pattern.append(vector)
    Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
    for index in seq:
         vector = [0 for _ in range(max_int)]
         vector[index] = 1
         pattern.append(vector)
    yenc.append(pattern)
    return Xenc, yenc

In [48]:
#Example of one hot encoding an integer encoded sequences.
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode 
alphabet=['0','1','2','3','4','5','6','7','8','9','+',' '] 
X, y = integer_encode(X, y, alphabet)
print(X, y)
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

[[5, 1]] [6]
['  5+1'] [' 6']
[[11, 11, 5, 10, 1]] [[11, 6]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]]


### 5 Sequence Generation Pipeline

We can tie all of these steps together into a function called generate data(), listed below. Given a designed number of samples, number of terms, the largest value of each term, and the alphabet of possible characters, the function will generate a set of input and output sequences.

In [49]:
#Function for generating a sequence, encoding and reshaping it for an LSTM model.
#Generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
    # generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    # return as NumPy arrays
    X, y = array(X), array(y)
    return X, y

### 6 Decode Sequences

Finally, we need to invert the encoding to convert the output vectors back into numbers so we can compare expected output integers to predicted integers. The invert() function below performs this operation. Key is first converting the binary encoding back into an integer using the argmax() function, then converting the integer back into a character using a reverse mapping of the integers to chars from the alphabet.

In [50]:
#Function for deciding an encoded input or output sequence.
#Invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return  ''.join(strings)

#### The network needs three configuration values defined by the specification of the addition problem:
1. n terms: The number of terms in the equation, (e.g. 2 for 10+10).
2. largest: The largest numerical value for each term (e.g. 10 for values between 1-10).
3. alphabet: The symbols used to encode the input and output sequences (e.g. 0-9, + and ‘ ’)

4. n chars: The size of the alphabet for a single time step (e.g. 12 for 0-9, ‘+’ and ‘ ’).
5. n in seq length: The number of time steps of encoded input sequences (e.g. 8 for
‘10+10+10’).
6. n out seq length: The number of time steps of an encoded output sequence (e.g. 2 for ‘30’)

We are now ready to define the Encoder-Decoder LSTM. We will use a single LSTM layer for the encoder and another single layer for the decoder.

The encoder is defined with 75 memory cells and the decoder with 50 memory cells. 

The output layer uses the categorical log loss for the 12 possible classes that may be predicted.

In [62]:
# number of math terms
n_terms = 2
# largest value for any single input digit
largest = 10
# scope of possible symbols for each input or output time step
alphabet = [str(x) for x in range(10)] + ['+', ' ']
# size of alphabet: (12 for 0-9, + and    )
n_chars = len(alphabet)
# length of encoded input sequence (5 for 2 terms  10+10)
n_in_seq_length = 5
# length of encoded output sequence (2 for  30 )
n_out_seq_length = 2

In [82]:
# define LSTM
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation= 'softmax' )))
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
model.summary()
# fit LSTM
X, y = generate_data(75000, n_terms, largest, alphabet)
model.fit(X, y, epochs=100, batch_size=32)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 75)                26400     
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 2, 75)             0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 2, 50)             25200     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 2, 12)             612       
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 2.4811 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 2.4638 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] -

Epoch 75/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 76/100
1/1 [==============================] - 0s 22ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 77/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 78/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 79/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 80/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 81/100
1/1 [==============================] - 0s 15ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 83/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 84/100
1/1 [==============================] - 0s 16ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 85/1

In [83]:
# evaluate LSTM
X, y = generate_data(100, n_terms, largest, alphabet)
loss, acc = model.evaluate(X, y, verbose=0)
print("Loss: %f, Accuracy: %f  "% (loss, acc*100))

Loss: 6.431972, Accuracy: 50.000000  


In [86]:
# predict
for _ in range(1):
    # generate an input-output pair
    X, y = generate_data(1, n_terms, largest, alphabet)
    # make prediction
    yhat = model.predict(X, verbose=0)
    # decode input, expected and predicted
    in_seq = invert(X[0], alphabet)
    out_seq = invert(y[0], alphabet)
    predicted = invert(yhat[0], alphabet)
    print("%s = %s (expect %s)"  % (in_seq, predicted, out_seq))

  5+7 = 11 (expect 12)
